In [ ]:
!pip3 install setuptools==64.0.2
!apt-get install swig
!pip3 install wrds
!pip3 install git+https://github.com/AI4Finance-LLC/FinRL-Library.git

In [ ]:
import os
import datetime

import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.use('Agg')

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
# 	date	open	high	low	close	volume	tic	day

import yfinance as yf

from finrl.meta.preprocessor.preprocessors import FeatureEngineer
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv

from stable_baselines3 import A2C, DDPG, PPO

In [ ]:
tickers_list = ['NVDA', 'AAPL', 'GOOG']  # list of tickers

# Define start and end dates
end_date = datetime.date.today()
start_date = end_date - datetime.timedelta(days=60)

# Download data for each ticker in the list
data = {}
for ticker in tickers_list:
    data[ticker] = yf.download(ticker, start=start_date, end=end_date)

# Convert data to dataframe with required format
dfs = []
for ticker in data.keys():
    df = data[ticker].reset_index()
    df['tic'] = ticker
    dfs.append(df)
df = pd.concat(dfs, ignore_index=True)
df.columns = df.columns.str.lower()
df = df[['date', 'open', 'high', 'low', 'close', 'volume', 'tic']]
df['date'] = df['date'].dt.strftime('%Y-%m-%d')

df.head()


In [ ]:
INDICATORS = ['macd',
               'rsi_30',
               'cci_30',
               'dx_30']

# create a feature engineer object
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list=INDICATORS,
                     use_turbulence=False,
                     user_defined_feature=False)

# Preprocess the current stock prices data
preprocessed = fe.preprocess_data(df)
preprocessed = preprocessed.fillna(0)
preprocessed = preprocessed.replace(np.inf,0)

preprocessed.sample(5)

TODO:

In [ ]:
# create a stock trading environment
stock_dimension = len(preprocessed.tic.unique())
state_space = 1 + 2*stock_dimension + len(tickers_list)*stock_dimension
env_kwargs = {
    "hmax": 100, 
    "initial_amount": 1000000,
    "num_stock_shares": len(tickers_list),
    "buy_cost_pct": 0.001, 
    "sell_cost_pct": 0.001, 
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension, 
    "reward_scaling": 1e-4,
    "print_verbosity":5
}

e_trade_gym = StockTradingEnv(df=preprocessed, **env_kwargs)

In [ ]:
# set the directory where the models are located
model_dir = 'trained_models'

# load the pre-trained models
model_names = ['A2C_0k_126', 'A2C_0k_189', 'A2C_0k_252', 'A2C_0k_315', 'DDPG_0k_126', 'DDPG_0k_189', 'DDPG_0k_252', 'DDPG_0k_315', 'PPO_0k_126', 'PPO_0k_189', 'PPO_0k_252', 'PPO_0k_315']
models = {}
for name in model_names:
    if name.startswith('A2C'):
        model_path = os.path.join(model_dir, name)
        model = A2C.load(model_path)
    elif name.startswith('DDPG'):
        model_path = os.path.join(model_dir, name)
        model = DDPG.load(model_path)
    elif name.startswith('PPO'):
        model_path = os.path.join(model_dir, name)
        model = PPO.load(model_path)
    models[name] = model

models

In [ ]:
# run the loaded models on the environment and get the portfolio value
for model_name, model in models.items():
    env = e_trade_gym
    obs = env.reset()
    done = False
    while not done:
        action, _states = model.predict(obs)
        obs, reward, done, info = env.step(action)
    print(f"{model_name} Sharpe Ratio: {info['sharpe_ratio']:.2f}")
